### Import packages:

In [1]:
import dva_sdhdf_combine
import imp
import os
import subprocess
import h5py
import numpy as np
from astropy.time import Time
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import datetime
import matplotlib.dates as mdates
from matplotlib.dates import HourLocator as HourLocator
from matplotlib.dates import MinuteLocator as MinuteLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy import units as u
from astropy.time import TimeDelta

#### Change the directory to where the files are located" ####
directory = '/media/ordoga/DVA_data/Mini_survey/'
##############################################################

### Read in the file listing the scan start and stop times:

In [2]:
scan_num = []    # The scan id number
scan_start = []  # Start time of the scan (UTC)
scan_stop = []   # Stop time of the scan (UTC)

# Read in the data and store it in arrays:
with open(directory+'azscans_out_mini_survey_good.txt') as fp:
    for line in fp:       
        scan_num.append(int(line.split()[0]))
        scan_start.append(line.split()[1]+'T'+line.split()[2][0:12])
        scan_stop.append(line.split()[3]+'T'+line.split()[4][0:12])

# Print out the scan numbers with their start and stop times:
for i in range(0,len(scan_num)):
    print(scan_num[i],scan_start[i],scan_stop[i])

# Convert start and stop times to Modified Julian Day (MJD).
# This is needed for plotting and for selecting out data collected
# between particular times:
scan_start_mjd = Time(scan_start, format='isot',scale='utc').mjd
scan_stop_mjd  = Time(scan_stop,  format='isot',scale='utc').mjd

2 2022-05-18T02:19:57.799 2022-05-18T02:37:59.622
3 2022-05-18T02:42:23.768 2022-05-18T03:00:25.975
4 2022-05-18T03:04:50.203 2022-05-18T03:22:52.085
5 2022-05-18T03:27:16.790 2022-05-18T03:45:18.698
6 2022-05-18T03:49:43.137 2022-05-18T04:07:45.238
7 2022-05-18T04:12:09.273 2022-05-18T04:30:11.468
8 2022-05-18T04:34:35.727 2022-05-18T04:52:37.999
9 2022-05-18T04:57:02.029 2022-05-18T05:15:04.069
10 2022-05-18T05:19:28.248 2022-05-18T05:37:30.300
11 2022-05-18T05:41:54.444 2022-05-18T05:59:56.440
12 2022-05-18T06:04:20.926 2022-05-18T06:22:22.771
13 2022-05-18T06:26:47.173 2022-05-18T06:44:49.373
14 2022-05-18T06:49:13.296 2022-05-18T07:07:15.002
15 2022-05-18T07:11:39.591 2022-05-18T07:29:41.713
16 2022-05-18T07:34:06.119 2022-05-18T07:52:07.963
17 2022-05-18T07:56:32.414 2022-05-18T08:14:34.333
18 2022-05-18T08:18:58.755 2022-05-18T08:37:00.654
19 2022-05-18T08:41:24.975 2022-05-18T08:59:27.324
20 2022-05-18T09:03:51.078 2022-05-18T09:21:52.714
21 2022-05-18T09:26:17.720 2022-05-18T0

### Read in scan files and stitch them together:
#### Note: this will take several minutes to run if stitching together all 3 days of data

In [3]:
t_set = []
az_set = []
dec_set = []
ra_set = []
el_set = []

# Use one of the scans to get the list of frequencies:
file = h5py.File(directory+'Mini_survey_scan_002.h5','r')
freq = file['data']['beam_0']['band_SB0']['frequency'][:]/1e6

# Create empty arrays for the power data:
RR_set = np.empty([0,len(freq)])
LL_set = np.empty([0,len(freq)])
reRL_set = np.empty([0,len(freq)])
imRL_set = np.empty([0,len(freq)])

# Loop through all the scans in the "scan_num" list:
for i in scan_num:
    
    # select the file:
    file = h5py.File(directory+'Mini_survey_scan_'+f"{i:03}"+'.h5','r')
    print(file)
    
    # access the correct location in the file structure:
    dataset = file['data']['beam_0']['band_SB0']['scan_0']
    
    # Add the position and time data to the corresponding arrays:
    dec_set = np.concatenate([dec_set,dataset['metadata']['declination']])
    ra_set = np.concatenate([ra_set,dataset['metadata']['right_ascension']])
    el_set = np.concatenate([el_set,dataset['metadata']['elevation']])
    az_set = np.concatenate([az_set,dataset['metadata']['azimuth']])
    t_set = np.concatenate([t_set,dataset['metadata']['utc']])
    
    # Add the spectrometer power data to the corresponding arrays:
    RR_set = np.concatenate([RR_set,dataset['data'][:,0,:]],axis=0)
    LL_set = np.concatenate([LL_set,dataset['data'][:,1,:]],axis=0)
    reRL_set = np.concatenate([reRL_set,dataset['data'][:,2,:]],axis=0)
    imRL_set = np.concatenate([imRL_set,dataset['data'][:,3,:]],axis=0)
    

<HDF5 file "Mini_survey_scan_002.h5" (mode r)>
<HDF5 file "Mini_survey_scan_003.h5" (mode r)>
<HDF5 file "Mini_survey_scan_004.h5" (mode r)>
<HDF5 file "Mini_survey_scan_005.h5" (mode r)>
<HDF5 file "Mini_survey_scan_006.h5" (mode r)>
<HDF5 file "Mini_survey_scan_007.h5" (mode r)>
<HDF5 file "Mini_survey_scan_008.h5" (mode r)>
<HDF5 file "Mini_survey_scan_009.h5" (mode r)>
<HDF5 file "Mini_survey_scan_010.h5" (mode r)>
<HDF5 file "Mini_survey_scan_011.h5" (mode r)>
<HDF5 file "Mini_survey_scan_012.h5" (mode r)>
<HDF5 file "Mini_survey_scan_013.h5" (mode r)>
<HDF5 file "Mini_survey_scan_014.h5" (mode r)>
<HDF5 file "Mini_survey_scan_015.h5" (mode r)>
<HDF5 file "Mini_survey_scan_016.h5" (mode r)>
<HDF5 file "Mini_survey_scan_017.h5" (mode r)>
<HDF5 file "Mini_survey_scan_018.h5" (mode r)>
<HDF5 file "Mini_survey_scan_019.h5" (mode r)>
<HDF5 file "Mini_survey_scan_020.h5" (mode r)>
<HDF5 file "Mini_survey_scan_021.h5" (mode r)>
<HDF5 file "Mini_survey_scan_022.h5" (mode r)>
<HDF5 file "M

### Make a degrees (instead of hours) version of RA, and an MJD version of the time array:

In [4]:
ra_set_deg = ra_set*360./24.
t_plt = Time(t_set, format='isot',scale='utc').mjd

### Read in the times when the noise source was turned on, and mask this out in the power data:

In [5]:
#### Change the directory to where the files are located" ####
directory = '/media/ordoga/DVA_data/Mini_survey/'
##############################################################

t_noise1 = []
t_noise2 = []

with open(directory+'noise_mini_survey_part1.txt') as fp:
    for line in fp:       
        t_noise1.append(line.split()[0]+'T'+line.split()[1][0:12])
        t_noise2.append(line.split()[2]+'T'+line.split()[3][0:12])

t_noise1_plt = Time(t_noise1, format='isot',scale='utc').mjd
t_noise2_plt = Time(t_noise2, format='isot',scale='utc').mjd

# Buffer time (in seconds) around data points where noise source
# is being turned on or off:
t_buff = 1./(3600.*24.)

# Make copies of the arrays in which noise will be masked out:
RR_set_clean = RR_set.copy()
LL_set_clean = LL_set.copy()
reRL_set_clean = reRL_set.copy()
imRL_set_clean = imRL_set.copy()

# Loop through times when noise source was turned on and replace
# data values with NaN at those times:
for i in range(0,len(t_noise1)):
    wnoise_sub_rem = np.where( (t_plt >= t_noise1_plt[i]-t_buff) & (t_plt <= t_noise2_plt[i]+t_buff) )[0]
    RR_set_clean[wnoise_sub_rem,:] = np.nan
    LL_set_clean[wnoise_sub_rem,:] = np.nan
    reRL_set_clean[wnoise_sub_rem,:] = np.nan
    imRL_set_clean[wnoise_sub_rem,:] = np.nan

### Plot the scans in RA/dec at a particular frequency:

In [ ]:
### Pick frequency to plot: ############
freq_plt = 800. # in MHz
########################################

df = freq[1]-freq[0]
wf = np.where(abs(freq-freq_plt)<df)[0][0]

fs = 12
fig1, axs = plt.subplots(1,1,figsize=(16,6))

axs.scatter(ra_set_deg, dec_set,s=2,c=10.*np.log10(LL_set_clean[:,wf]),cmap='viridis',vmin=70.5, vmax=71.5)
axs.set_xlabel('RA (deg.)',fontsize=fs)
axs.set_ylabel('dec (deg.)',fontsize=fs)
axs.tick_params(axis="x", labelsize=fs)
axs.tick_params(axis="y", labelsize=fs)
axs.set_xlim(360,0)
axs.set_ylim(0,90)
axs.set(aspect='equal')
axs.grid()

### Plot a particular scan at a particular frequency:

In [ ]:
### Pick a scan id to plot: #############
scan_plot = 13
freq_plt = 800. # in MHz
########################################

df = freq[1]-freq[0]
wf = np.where(abs(freq-freq_plt)<df)[0][0]

w = np.where(np.array(scan_num) == scan_plot)[0][0]
t1 = scan_start[w]
t2 = scan_stop[w]
t1_plt = scan_start_mjd[w]
t2_plt = scan_stop_mjd[w]

print('Plotting scan ',scan_plot,' with start and end times:')
print('start =',t1,' stop =',t2)

wplot = np.where((t_plt>=t1_plt) & (t_plt<=t2_plt))

fs = 12
fig1, axs = plt.subplots(1,1,figsize=(16,6))

axs.scatter(ra_set_deg[wplot], dec_set[wplot],s=2,c=10.*np.log10(LL_set_clean[wplot,wf]),
            cmap='viridis',vmin=70.5, vmax=71.5)
axs.set_xlabel('RA (deg.)',fontsize=fs)
axs.set_ylabel('dec (deg.)',fontsize=fs)
axs.tick_params(axis="x", labelsize=fs)
axs.tick_params(axis="y", labelsize=fs)
axs.set_xlim(360,0)
axs.set_ylim(0,90)
axs.set(aspect='equal')
axs.grid()